In [13]:
import os

def img2label_paths(img_paths):
    # Define label paths as a function of image paths
    # sa, sb = f'{os.sep}images{os.sep}', f'{os.sep}labels{os.sep}'  # /images/, /labels/ substrings #origin
    sa, sb = f'{os.sep}images{os.sep}', f'{os.sep}labels_yolo{os.sep}'  # /images/, /labels/ substrings
    
    return [sb.join(x.rsplit(sa, 1)).rsplit('.', 1)[0] + '.txt' for x in img_paths]

In [2]:
img_paths = os.listdir('/data/NIA50/50-2/data/NIA48/temp_data/images/')

In [1]:
import open3d as o3d
import cv2
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
# device = o3d.core.Device('cuda:1')
pcd_f = o3d.t.io.read_point_cloud('/data/NIA50/SUSTechPOINTS_2-050/data/example/lidar/000965.pcd')
print(pcd_f)

positions = pcd_f.point.positions.numpy()
intensity = pcd_f.point.intensity.numpy()
pcd = np.concatenate((positions, intensity), axis = 1)

xyzi = np.stack([pcd[:,1]*-1, pcd[:, 0], pcd[:, 2], pcd[:, 3]], axis=1)

# xyz = xyzi[:, :3]
# i = [[i] for i in xyzi[:, 3]]

# pcd = o3d.t.geometry.PointCloud()
# pcd.point["positions"] = o3d.core.Tensor(xyz)
# pcd.point["intensity"] = o3d.core.Tensor(i)

# o3d.t.io.write_point_cloud('/data/NIA50/SUSTechPOINTS_2-050/data/example copy/lidar/000965.pcd', pcd, write_ascii=True)
# np.frombuffer(pcd.tobytes(), dtype='float32').reshape(-1, 4) # bytes에서 numpy로 복원

: 

: 

In [1]:
def lidar_camera_calib(json_name, camera_mat, dist_coeff, lidar_points, image_points):
    retval, rvec, tvec, inliers = cv2.solvePnPRansac(lidar_points, image_points, camera_mat, dist_coeff, iterationsCount=5000, reprojectionError=8.0)
    print("-----")
    rot_mat, jac = cv2.Rodrigues(rvec)
    print(rot_mat)
    print("-----")
    print(tvec)
    print("-----")
    rt = np.hstack((rot_mat, tvec))
    T = np.vstack((rt, [[0,0,0,1]]))
    print (T)
    calib_json = {
        "extrinsic": T.flatten().tolist(),
        "intrinsic": camera_mat.flatten().tolist()
    }
    json_object = json.dumps(calib_json, indent = 4) 
    # Writing to sample.json 
    with open(json_name, "w") as outfile: 
        outfile.write(json_object) 
    return T

In [4]:
import numpy as np
import glob, cv2

# 종료 기준(termination criteria)를 정한다.
criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30 ,0.001)

# Object Point(3D)를 준비한다. (0,0,0),(1,0,0),(2,0,0)... 처럼
objp = np.zeros((6*7,3),np.float32)
# np,mgrid[0:7,0:6]으로 (2,7,6) 배열 생성
# Transpose 해줘서 (6,7,2)로, reshpae(-1,2)로 flat 시켜서 (42,2)로 변환
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# 이미지로 부터의 Object point와 Image points를 저장하기 위한 배열
objpoints = [] # 실제 세계의 3D 점들 
imgpoints = [] # 2D 이미지의 점들

# 전체 path를 받기 위해 os말고 glob 사용
images = glob.glob('./images/Calib/*.jpg')

for name in images:
    img = cv2.imread(name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 체스판의 코너들 찾기
    ret, corners = cv2.findChessboardCorners(gray,(7,6),None)

    # 찾았으면, Object points, Image points 추가하기 (이후에 수정한다)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # 코너를 그리고 봐보자
        img = cv2.drawChessboardCorners(img,(7,6),corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(2000)
cv2.destroyAllWindows()

In [5]:
criteria

(3, 30, 0.001)

In [24]:
import open3d as o3d
import numpy as np
import cv2
import torch
import pandas as pd
import re
import pickle as pkl
import json
import glob
import shutil
import os
from sklearn.model_selection import train_test_split
from scipy.spatial.transform import Rotation as R


# 문자열 숫자리스트로 바꾸는 함수
def str2list(txt):
    txt = txt.replace('\n', '').split(',')
    txt = list(map(float, txt))
    
    return txt


# 리스트를 문자열로 바꾸는 함수
def list2str(list):
    list = ' '.join(map(str, list))
    
    return list


# alpha 구하는 공식
import math

def normalizeAngle(angle):
    result = angle % (2*math.pi)
    if result < -math.pi:
        result += 2*math.pi
    elif result > math.pi:
        result -= 2*math.pi
    return result

def cal_alpha_ori(x, z, ry):  
    angle = ry
    angle -= -math.atan2(z, x) -1.5*math.pi 
    alpha = normalizeAngle(angle)
    return alpha # -1.818032754845337
# cal_alpha_ori(2.5702, 9.7190, -1.5595)

In [25]:
with open('/data/hwang/datasets/kitti/training/calib/000055.txt', 'r') as f:
    txt = f.readlines()

p0 = txt[0].replace('\n', '').split(' ')[1:] # 0번 카메라, projection matrix (= intrinsic * extrinsic)
p1 = txt[1].replace('\n', '').split(' ')[1:] # 1번 카메라
p2 = txt[2].replace('\n', '').split(' ')[1:] # 2번 카메라 (모델에 이 정보를 사용), (i, 4)위치는 기본 카메라와의 위상 차이 (각 x, y, z 축)
p3 = txt[3].replace('\n', '').split(' ')[1:] # 3번 카메라
R_rect = txt[4].replace('\n', '').split(' ')[1:] # 스테레오 카메라로 촬영 시 rotation matrix 보정 수치 (모노 카메라면 단위행렬 사용)
Tr_velo = txt[5].replace('\n', '').split(' ')[1:] # extrinsic (사용은 rigid body transformation 형태로 사용, extrinsic 4행에 [0, 0, 0, 1]을 추가)
Tr_imu = txt[6].replace('\n', '').split(' ')[1:]

print(np.around(np.asarray(p0, dtype=float).reshape(-1, 4), 1), '\n')
print(np.around(np.asarray(p1, dtype=float).reshape(-1, 4), 1), '\n')
print(np.around(np.asarray(p2, dtype=float).reshape(-1, 4), 1), '\n')
print(np.around(np.asarray(p3, dtype=float).reshape(-1, 4), 1), '\n')
print(np.asarray(R_rect, dtype=float).reshape(-1, 3), '\n')
print(np.asarray(Tr_velo, dtype=float).reshape(-1, 4), '\n')
print(np.asarray(Tr_imu, dtype=float).reshape(-1, 4), '\n')

[[721.5   0.  609.6   0. ]
 [  0.  721.5 172.9   0. ]
 [  0.    0.    1.    0. ]] 

[[ 721.5    0.   609.6 -387.6]
 [   0.   721.5  172.9    0. ]
 [   0.     0.     1.     0. ]] 

[[7.215e+02 0.000e+00 6.096e+02 4.490e+01]
 [0.000e+00 7.215e+02 1.729e+02 2.000e-01]
 [0.000e+00 0.000e+00 1.000e+00 0.000e+00]] 

[[ 721.5    0.   609.6 -339.5]
 [   0.   721.5  172.9    2.2]
 [   0.     0.     1.     0. ]] 

[[ 0.9999239   0.00983776 -0.00744505]
 [-0.0098698   0.9999421  -0.00427846]
 [ 0.00740253  0.00435161  0.9999631 ]] 

[[ 7.533745e-03 -9.999714e-01 -6.166020e-04 -4.069766e-03]
 [ 1.480249e-02  7.280733e-04 -9.998902e-01 -7.631618e-02]
 [ 9.998621e-01  7.523790e-03  1.480755e-02 -2.717806e-01]] 

[[ 9.999976e-01  7.553071e-04 -2.035826e-03 -8.086759e-01]
 [-7.854027e-04  9.998898e-01 -1.482298e-02  3.195559e-01]
 [ 2.024406e-03  1.482454e-02  9.998881e-01 -7.997231e-01]] 



In [3]:
from PIL import Image

im = Image.open('/data/NIA50/SUSTechPOINTS_2-050/data/kitti/camera/front/000055.png').convert('RGB')

im.save('/data/NIA50/SUSTechPOINTS_2-050/data/kitti/camera/front/000055.jpg', 'jpeg')